# Monte Carlo method
This code calculates de value and the delta for any option/derivative given its payoff function, using Monte Carlo simulation.<br><br>

### Index
<h4 style="margin-bottom:5px">(1) Path-independent options</h4>
&nbsp;&nbsp;&nbsp;&nbsp; Contains two functions that calculate the option value and option delta. The payoff functions defined only take as input the final value of the equity random walk at each iteration.

<h4 style="margin-bottom:5px">(2) Path-dependent options</h4>
&nbsp;&nbsp;&nbsp;&nbsp; Two functions calculate the option value and option delta. The payoff functions defined take as input the entire random walk of the equity at each iteration.

## (1) Path-independent options

In [ ]:
import math
import numpy as np

def European_call_payoff(S):
    return max(S - 110, 0)

def MC_option(payoff, S, T, sigma, r, M, t = 0):
    '''
    Estimates the value of any non-path-dependent option (defined by its payoff function) using Monte Carlo method.

    Parameters:
        payoff (function): Payoff function of the option.
        S (float): Current asset value.
        T (float): Time of maturity (Time to maturity if t = 0).
        sigma (float): Volatility.
        r (float): Risk-free rate.
        M (int): Number of simulations to run for Monte Carlo (higher number, higher precision).
        t (float): Current time.
    
    Returns:
        float: Option value at S and t.
    '''
    # Run simulations
    total = 0
    for i in range(M):
        X = np.random.normal(scale = math.sqrt(T - t)) # Wiener process
        # Using Wiener process and its negative, compute 2 stock evolutions (antithetic variables method, to speed up convergence)
        S_T = S * math.exp((r - sigma ** 2 / 2) * (T - t) + sigma * X) # Geometric Brownian Motion solution
        S_T_n = S * math.exp((r - sigma ** 2 / 2) * (T - t) + sigma * (- X)) # Geometric Brownian Motion solution
        total = total + payoff(S_T) + payoff(S_T_n) # Add payoff of this simulation to the sum, to calculate average

    return math.exp(-r * (T - t)) * total / (2 * M) # Discount to present value


def MC_delta(payoff, S, T, sigma, r, M, t = 0):
    '''
    Estimates the Delta of any non-path-dependent option (defined by its payoff function) using Monte Carlo method.

    Parameters:
        payoff (function): Payoff function of the option.
        S (float): Current asset value.
        T (float): Time of maturity (Time to maturity if t = 0).
        sigma (float): Volatility.
        r (float): Risk-free rate.
        M (int): Number of simulations to run for Monte Carlo (higher number, higher precision).
        t (float): Current time.
    
    Returns:
        float: Delta of the option at S and t.
    '''
    # Run simulations
    epsilon = 1e-10
    total_plus, total_minus = 0, 0
    for i in range(M):
        X = np.random.normal(scale = math.sqrt(T - t)) # Wiener process

        # Using Wiener process and its negative, compute 2 stock evolutions (antithetic variables method, to speed up convergence)
        S_T = S * math.exp((r - sigma ** 2 / 2) * (T - t) + sigma * X) # Geometric Brownian Motion solution
        S_T_n = S * math.exp((r - sigma ** 2 / 2) * (T - t) + sigma * (- X)) # Geometric Brownian Motion solution

        # Add payoff of this simulation to the sums, to calculate average
        total_plus = total_plus + payoff(S_T * (1 + epsilon / S)) + payoff(S_T_n * (1 + epsilon / S)) # Total at S + epsilon
        total_minus = total_minus + payoff(S_T * (1 - epsilon / S)) + payoff(S_T_n * (1 - epsilon / S)) # Total at S - epsilon

    # Compute option value at S + epsilon and S - epsilon
    V_plus = math.exp(-r * (T - t)) * total_plus / (2 * M) # Discount to present value
    V_minus = math.exp(-r * (T - t)) * total_minus / (2 * M) # Discount to present value

    # Return delta approximation
    return (V_plus - V_minus) / (2 * epsilon)

# European call 
print(MC_option(European_call_payoff, S=100, T=1, sigma=0.2, r=0.05, M=1000000))

print(MC_delta(European_call_payoff, S=100, T=1, sigma=0.2, r=0.05, M=1000000))

6.0285967065908315
0.4497024974625674


## (2) Path-dependent options

In [ ]:
from functools import partial

def MC_option(payoff, S, T, sigma, r, dt, M, t = 0):
    '''
    Estimates the value of any path-dependent option (defined by its payoff function) using Monte Carlo method.

    Parameters:
        payoff (function): Payoff function of the option, which takes the entire path as input.
        S (float): Current asset value.
        T (float): Time of maturity (Time to maturity if t = 0).
        sigma (float): Volatility.
        r (float): Risk-free rate.
        dt (float): Time step.
        M (int): Number of simulations to run for Monte Carlo (higher number, higher precision).
        t (float): Current time.
    
    Returns:
        float: Option value at S and t.
    '''
    N = round(T / dt) # Total time steps
    step = np.arange(N + 1) # Step array

    # Run simulations
    total = 0
    for i in range(M):
        # Compute Wiener process
        X_1 = np.zeros(N + 1)
        for j in range(N):
            dX = np.random.normal(scale = math.sqrt(dt))
            X_1[j + 1] = X_1[j] + dX

        # Using Wiener process and its negative, compute 2 stock evolutions (antithetic variables method, to speed up convergence)
        S_motion = S * np.exp((r - sigma ** 2 / 2) * (step * dt) + sigma * X_1)
        S_motion_n = S * np.exp((r - sigma ** 2 / 2) * (step * dt) + sigma * (- X_1))

        total = total + payoff(S_motion) + payoff(S_motion_n)
        
    return math.exp(-r * T) * total / (2 * M)


def MC_delta(payoff, S, T, sigma, r, dt, M, t = 0):
    '''
    Estimates the Delta of any path-dependent option (defined by its payoff function) using Monte Carlo method.

    Parameters:
        payoff (function): Payoff function of the option, which takes the entire path as input.
        S (float): Current asset value.
        T (float): Time of maturity (Time to maturity if t = 0).
        sigma (float): Volatility.
        r (float): Risk-free rate.
        dt (float): Time step.
        M (int): Number of simulations to run for Monte Carlo (higher number, higher precision).
        t (float): Current time.
    
    Returns:
        float: Delta of the option at S and t.
    '''
    N = round(T / dt) # Total time steps
    step = np.arange(N + 1) # Step array

    # Run simulations
    epsilon = 1e-10
    total_plus, total_minus = 0, 0
    for i in range(M):
        # Compute Wiener process
        X_1 = np.zeros(N + 1)
        for j in range(N):
            dX = np.random.normal(scale = math.sqrt(dt))
            X_1[j + 1] = X_1[j] + dX

        # Using Wiener process and its negative, compute 2 stock paths for each initial stock value (antithetic variables method, to speed up convergence)
        S_motion_plus = (S + epsilon) * np.exp((r - sigma ** 2 / 2) * (step * dt) + sigma * X_1) # Stock path starting at S + epsilon
        S_motion_minus = (S - epsilon) * np.exp((r - sigma ** 2 / 2) * (step * dt) + sigma * X_1) # Stock path starting at S - epsilon

        S_motion_n_plus = (S + epsilon) * np.exp((r - sigma ** 2 / 2) * (step * dt) + sigma * (- X_1)) # Starting at S + epsilon
        S_motion_n_minus = (S - epsilon) * np.exp((r - sigma ** 2 / 2) * (step * dt) + sigma * (- X_1)) # Starting at S - epsilon

        total_plus = total_plus + payoff(S_motion_plus) + payoff(S_motion_n_plus)
        total_minus = total_minus + payoff(S_motion_minus) + payoff(S_motion_n_minus)
    
    # Compute stock value at S + epsilon and S - epsilon
    V_plus = math.exp(-r * T) * total_plus / (2 * M) # Discount to present value
    V_minus = math.exp(-r * T) * total_minus / (2 * M) # Discount to present value

    # Return delta approximation
    return (V_plus - V_minus) / (2 * epsilon)



# Asian option
def Asian_payoff(S_array, op_class, strike):
    '''
    Payoff function of an Asian call option.

    Parameters:
        S_array (np.ndarray): Stock path array
    
    Returns:
        float: Payoff of the option, given the stock path
    '''
    if op_class == "C":
        return max(np.mean(S_array) - strike, 0)
    elif op_class == "P":
        return max(strike - np.mean(S_array), 0)
    else:
        raise ValueError(f'Please introduce a valid op_class: either "C" (Call) or "P" (Put)')

# Since inside MC_option or MC_delta, the payoff function only expects S_array as an argument, customize the rest of the parameters by making a partial.
asian_payoff_partial = partial(
    Asian_payoff,
    op_class="C",
    strike=110,
    )

print(MC_option(asian_payoff_partial, S=100, T=1, sigma=0.2, r=0.05, dt=1/52, M=50000))

print(MC_delta(asian_payoff_partial, S=100, T=1, sigma=0.2, r=0.05, dt=1/52, M=50000))


# Barrier option
def Barrier_payoff(S_array, kick, op_class, strike, upper_barrier = np.inf, lower_barrier = 0):
    '''
    Payoff function of a barrier option.

    Parameters:
        S_array (np.ndarray): Stock path array.
        kick (str): Is it a kick-in or a kick-out barrier?
        op_class (str): Call or put?
        strike (float)
        upper_barrier (float)
        lower_barrier (float)
    
    Returns:
        float: Payoff of the option.
    '''
    # Option activates if it's a kick-in and the stock has crossed the barrier, or if it's a kick-out and the stock hasn't.
    if (kick == "In" and (max(S_array) > upper_barrier or min(S_array) < lower_barrier)) or (kick == "Out" and (max(S_array) < upper_barrier or min(S_array) > lower_barrier)):
        
        if op_class == "C":
            return max(S_array[-1] - strike, 0)
        elif op_class == "P":
            return max(strike - S_array[-1], 0)
        else:
            raise ValueError(f'Please introduce a valid op_class: either "C" (Call) or "P" (Put)')
    
    elif kick == "Out" or kick == "In":
        # If option is desactivated, return 0 (assuming correct kick argument)
        return 0
    else:
        raise ValueError(f'Please introduce a valid kick: either "In" (Kick-in) or "Out" (Kick-out)')

# Since inside MC_option or MC_delta, the payoff function only expects S_array as an argument, customize the rest of the parameters by making a partial.
barrier_payoff_partial = partial(
    Barrier_payoff,
    kick="In",
    op_class="C",
    strike=110,
    upper_barrier=120,
    lower_barrier=80
    )

print(MC_option(barrier_payoff_partial, S=100, T=1, sigma=0.2, r=0.05, dt=1/365, M=10000))

print(MC_delta(barrier_payoff_partial, S=100, T=1, sigma=0.2, r=0.05, dt=1/365, M=10000))


# Lookback option
def Lookback_payoff(S_array, op_class):
    if op_class == "C":
        return S_array[-1] - min(S_array)
    elif op_class == "P":
        return max(S_array) - S_array[-1]
    else:
        raise ValueError(f'Please introduce a valid op_class: either "C" (Call) or "P" (Put)')

# Since inside MC_option or MC_delta, the payoff function only expects S_array as an argument, customize the rest of the parameters by making a partial.
lookback_payoff_partial = partial(
    Lookback_payoff,
    op_class="C",
    )

print(MC_option(lookback_payoff_partial, S=100, T=1, sigma=0.2, r=0.05, dt=1/365, M=10000))

print(MC_delta(lookback_payoff_partial, S=100, T=1, sigma=0.2, r=0.05, dt=1/365, M=10000))

1.970853325165073
0.2840472301812724
5.83396162460822
0.3965139327988254
16.664675284468398
0.16624923659946944
